In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time  # 导入 time 模块
from selenium.webdriver.common.by import By
import re

/Users/yupeili/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
!pip3 install webdriver-manager
!pip3 install selenium

Defaulting to user installation because normal site-packages is not writeable
  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
  Using cached selenium-4.25.0-py3-none-any.whl (9.7 MB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
url = 'https://icd.who.int/browse/2024-01/mms/en'

In [3]:
pattern = re.compile(
    r'\bdepress(?:ion|ive|ed|es|ing)?s?\b',
    re.IGNORECASE
)
results = pd.DataFrame(columns=['Title', 'Description'])
#keywords = 'depress'
keywords = 'anxiety'

In [4]:
service = Service(ChromeDriverManager().install())

In [5]:
browser = webdriver.Chrome(service=service)


In [6]:
browser.get(url)

In [30]:
#titles = browser.find_elements(By.CSS_SELECTOR, 'span.titleinh')
span_elements = browser.find_elements(By.CSS_SELECTOR, "span.collapsed")

# Loop through each element and click on it
for span in span_elements:
    try:
        span.click()  # Click on the span
        time.sleep(1)  # Optional: Add delay if needed
    except Exception as e:
        print(f"Could not click span: {e}")

Could not click span: Message: element click intercepted: Element <span class="collapsed">...</span> is not clickable at point (118, 673). Other element would receive the click: <div class="browserBodyLeft hierarchyResizable">...</div>
  (Session info: chrome=130.0.6723.60)
Stacktrace:
0   chromedriver                        0x000000010284f634 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x0000000102847e94 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x00000001022b4104 cxxbridge1$string$len + 88416
3   chromedriver                        0x00000001022fba74 cxxbridge1$string$len + 381648
4   chromedriver                        0x00000001022fa0dc cxxbridge1$string$len + 375096
5   chromedriver                        0x00000001022f8320 cxxbridge1$string$len + 367484
6   chromedriver                        0x00000001022f7534 cxxbridge1$string$len + 363920
7   chromedriver                        0x00000001022ece14 cxxbridge1$string$len + 32113

KeyboardInterrupt: 

In [31]:
titles = browser.find_elements(By.CSS_SELECTOR, 'span.titleinh')


In [32]:
len(titles)

41175

In [33]:
count = 0
for i in titles:
    
    if keywords in i.text:
        #print(i.text)
        count += 1

count

70

In [34]:
#titles = browser.find_elements(By.CSS_SELECTOR, 'span.titleinh')
for title in titles[1:]:
    try:
        title_text = title.text
        #print(title_text)  # 这里也放入 try 块
    except Exception as e:
        print(f"Error occurred while accessing title text")
        continue

    # 点开所有# class="collapsed"
    if keywords in title_text:
    # TODO（可能要获得下级title）
    #if pattern.search(title_text):
        title.click()

        try:
            description_element = WebDriverWait(browser, 10).until(
                EC.visibility_of_element_located((By.CLASS_NAME, 'property'))
            )
            description_text = description_element.text  # 获取描述文本
            print(description_text)
            new_row = pd.DataFrame({'Title': [title_text], 'Description': [description_text]})
            results = pd.concat([results, new_row], ignore_index=True)  # 使用 concat
            
        except Exception as e:
            print(f"Error occurred while getting description for '{title_text}': {e}")

        time.sleep(1) 

print(results)


Bipolar type I disorder, current episode depressive, mild is diagnosed when the definitional requirements for Bipolar type I disorder have been met and the current episode is depressive at a mild level of severity. A depressive episode is characterised by a period of depressed mood or diminished interest in activities occurring most of the day, nearly every day during a period lasting at least two weeks accompanied by other symptoms such as difficulty concentrating, feelings of worthlessness or excessive or inappropriate guilt, hopelessness, recurrent thoughts of death or suicide, changes in appetite or sleep, psychomotor agitation or retardation, and reduced energy or fatigue. In a mild depressive episode, the individual is usually distressed by the symptoms and has some difficulty in continuing to function in one or more domains (personal, family, social, educational, occupational, or other important domains). There are no delusions or hallucinations during the episode.
Bipolar type 

In [35]:
output_file_path = "./axiety_data.csv"
results.to_csv(output_file_path, index=False)


In [ ]:
browser.quit()